# Lecture 14 Notes: Pipelines

### Load imports

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder, Normalizer, MaxAbsScaler, OneHotEncoder
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_selection import VarianceThreshold
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.metrics import confusion_matrix

In [2]:
# load income data
income_df = pd.read_csv('data/income_data.csv')

income_df.head()


,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


### Inspect the dataset

In [4]:
# look at number of each class
income_df['income'].value_counts()


income
<=50K    24720
>50K      7841
Name: count, dtype: int64

### Prepare data for machine learning

In [8]:
target_variable = 'income'

# Correctly identify numerical and categorical columns without trying to drop the target variable
numerical_cols = income_df.select_dtypes(include=['int64', 'float64']).columns
categorical_cols = income_df.select_dtypes(include=['object', 'category']).columns



# Now, ensure that 'income' is not included in numerical_cols or categorical_cols if it's not meant to be there
categorical_cols = [col for col in categorical_cols if col != 'income']

categorical_cols

['workclass',
 'education',
 'marital-status',
 'occupation',
 'relationship',
 'race',
 'sex',
 'native-country']

In [10]:
X = income_df.drop('income', axis=1)
y = income_df['income']

y

0        <=50K
1        <=50K
2        <=50K
3        <=50K
4        <=50K
         ...  
32556    <=50K
32557     >50K
32558    <=50K
32559    <=50K
32560     >50K
Name: income, Length: 32561, dtype: object

### Encode labels

In [11]:
# use label encoder to encode the target variable
encoder = LabelEncoder()

y_encoded = encoder.fit_transform(y)

y_encoded


array([0, 0, 0, ..., 0, 0, 1])

### Split data

In [12]:
# make training and testing data
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=1/3, random_state=42)

X_test

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country
14160,27,Private,160178,Some-college,10,Divorced,Adm-clerical,Not-in-family,White,Female,0,0,38,United-States
27048,45,State-gov,50567,HS-grad,9,Married-civ-spouse,Exec-managerial,Wife,White,Female,0,0,40,United-States
28868,29,Private,185908,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,Black,Male,0,0,55,United-States
5667,30,Private,190040,Bachelors,13,Never-married,Machine-op-inspct,Not-in-family,White,Female,0,0,40,United-States
7827,29,Self-emp-not-inc,189346,Some-college,10,Divorced,Craft-repair,Not-in-family,White,Male,2202,0,50,United-States
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18423,43,Private,90582,HS-grad,9,Married-civ-spouse,Transport-moving,Husband,White,Male,0,0,40,United-States
9043,21,Private,165107,Some-college,10,Never-married,Priv-house-serv,Own-child,White,Female,0,0,40,United-States
21845,34,Private,58305,HS-grad,9,Married-civ-spouse,Craft-repair,Husband,White,Male,0,1573,40,United-States
2146,18,Private,98044,11th,7,Never-married,Prof-specialty,Own-child,White,Female,0,0,40,United-States


### Initialize and train classifier

In [13]:
# Define preprocessing for numerical columns (scale them)
numerical_transformer = StandardScaler()


# Define preprocessing for categorical columns (encode them)
categorical_transformer = OneHotEncoder(handle_unknown='ignore')


# Create the preprocessing pipeline for both types of attributes
preprocessor = ColumnTransformer(
  transformers=[
    ('num', numerical_transformer, numerical_cols),
    ('cat', categorical_transformer, categorical_cols)
  ]
)

### Create pipeline and evaluate its accuracy

In [14]:
# define the pipeline
pipeline = Pipeline(
  steps=[
    ('preprocessor', preprocessor),
    ('classifier', KNeighborsClassifier())
  ]
)

# fit the pipeline
pipeline.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  Index(['age', 'fnlwgt', 'education-num', 'capital-gain', 'capital-loss',
       'hours-per-week'],
      dtype='object')),
                                                 ('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['workclass', 'education',
                                                   'marital-status',
                                                   'occupation', 'relationship',
                                                   'race', 'sex',
                                                   'native-country'])])),
                ('classifier', KNeighborsClassifier())])

In [16]:
print(f'Training score: {pipeline.score(X_train, y_train)}')

Training score: 0.8828949186898235


In [17]:
print(f'Testing score: {pipeline.score(X_test, y_test)}')

Testing score: 0.8306615072784227


### Look at confusion matrix

In [19]:
# get predictions
y_pred = pipeline.predict(X_test)

y_pred 

# confusion matrix
confusion_matrix(y_test, y_pred)


array([[7467,  814],
       [1024, 1549]])

### Run grid search to find best parameters

In [21]:
# Define the new pipeline
pipeline = Pipeline(
  steps=[
    ('preprocessor', preprocessor),
    ('selector', VarianceThreshold()),
    ('classifier', KNeighborsClassifier())
  ]
)

# Define the parameter grid to search
param_grid = {
  'selector__threshold': [0, 0.001, 0.01, 0.1],
  'classifier__n_neighbors': [3, 5, 7],
  'classifier__p': [1, 2]
}

# Setup and run the GridSearchCV
grid_search = GridSearchCV(pipeline, param_grid, cv=5)

grid_search.fit(X_train, y_train)


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('preprocessor',
                                        ColumnTransformer(transformers=[('num',
                                                                         StandardScaler(),
                                                                         Index(['age', 'fnlwgt', 'education-num', 'capital-gain', 'capital-loss',
       'hours-per-week'],
      dtype='object')),
                                                                        ('cat',
                                                                         OneHotEncoder(handle_unknown='ignore'),
                                                                         ['workclass',
                                                                          'education',
                                                                          'marital-status',
                                                                          'occupation',
                                                                          'relationship',
                                                                          'race',
                                                                          'sex',
                                                                          'native-country'])])),
                                       ('selector', VarianceThreshold()),
                                       ('classifier', KNeighborsClassifier())]),
             param_grid={'classifier__n_neighbors': [3, 5, 7],
                         'classifier__p': [1, 2],
                         'selector__threshold': [0, 0.001, 0.01, 0.1]})

In [22]:
# get the best parameters
grid_search.best_params_


{'classifier__n_neighbors': 7, 'classifier__p': 1, 'selector__threshold': 0.01}

### Evaluate optimized model

In [24]:
print(f'Training set score: {grid_search.score(X_train, y_train)}')
print(f'Testing set score: {grid_search.score(X_test, y_test)}')


Training set score: 0.872575666835583
Testing set score: 0.8370186106504515


In [25]:
# make confusion matrix
y_pred = grid_search.predict(X_test)

confusion_matrix(y_test, y_pred)

array([[7537,  744],
       [1025, 1548]])

# Activity

Follow the example in this [blog post](https://machinelearningmastery.com/modeling-pipeline-optimization-with-scikit-learn/) to make a pipeline for training a model to predict the location of enzymes for sequence metrics.